In [1]:
import pandas as pd
import numpy as np
import requests
import http.client, urllib.parse
from math import cos, asin, sqrt, pi
import datetime as datetime

In [2]:
### Get API key from positionstack
geo_api_key = '7c87122da92497c5685034ae9021644c'

# Buncha Functions

In [3]:
def find_lats(city, state):
    
    conn = http.client.HTTPConnection('api.positionstack.com')

    params = urllib.parse.urlencode({
        'access_key': geo_api_key,
        'query': city,
        'region': state,
        'limit': 1,
        })

    conn.request('GET', '/v1/forward?{}'.format(params))

    res = conn.getresponse()
    data = res.read().decode('utf-8')

    latitude = float(data.split('{')[2].split(',')[0].split(':')[1])
    longitude = float(data.split('{')[2].split(',')[1].split(':')[1])
    lat_lon = [latitude,longitude]
    
    return lat_lon

In [4]:
def recursive(lat, new_long, geo_api_key):
    conn = http.client.HTTPConnection('api.positionstack.com')
    params = urllib.parse.urlencode({
    'access_key': geo_api_key,
    'query': str(lat)+','+str(new_long),
    'limit': 1,
    })

    conn.request('GET', '/v1/reverse?{}'.format(params))

    res = conn.getresponse()
    data = res.read().decode('utf-8')
    town = data.split('{')[2].split(',')[4].split(':')[1]
    country = data.split('{')[2].split(',')[15].split(':')[1]
    label = data.split(':')[-1]
    
    return label, town, country

In [5]:
def whole_thing(city, state):
    now = datetime.datetime.now()
    five_pm = datetime.datetime(now.year,now.month,now.day,18,30,0)
    day_pct = ((five_pm-now).days*(24*60*60)+(five_pm-now).seconds)/(24*60*60)
    
    lat_lon = find_lats(city, state)
    new_long = lat_lon[1]+360*day_pct
    
    conn = http.client.HTTPConnection('api.positionstack.com')

    params = urllib.parse.urlencode({
        'access_key': geo_api_key,
        'query': str(lat_lon[0])+','+str(new_long),
        'limit': 1,
        })

    conn.request('GET', '/v1/reverse?{}'.format(params))

    res = conn.getresponse()
    data = res.read().decode('utf-8')
    town = data.split('{')[2].split(',')[4].split(':')[1]
    country = data.split('{')[2].split(',')[15].split(':')[1]
    label = data.split(':')[-1]
    
    ### Do the recursive thing now.
    new_lat = lat_lon[0]
    if country == 'null':
        adj = 1
        while country == 'null':
            new_lat = new_lat + 3*adj
            if new_lat>80:
                adj = adj * -1
                new_lat = lat_lon[0]
            if new_long<-80:
                print("Party in the Ocean")
                break
            label, town, country = recursive(new_lat, new_long, geo_api_key)

    print(label)
    print(new_lat, new_long)
    

# Run it!

In [8]:
whole_thing('Brooklyn', 'New York')

"Torrente Bilioso, Tricarico, MT, Italy"}]}
40.652256 16.251751333333345
